In [16]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchaudio 
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'
  torch.cuda.manual_seed_all(1)

/Users/Carlos/Documents/Anaconda/anaconda3/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [21]:
!mkdir augmented
!mkdir augmented/TRAIN
!mkdir augmented/TRAIN/covid
!mkdir augmented/TRAIN/no_covid
!mkdir augmented/TEST
!mkdir augmented/TEST/covid
!mkdir augmented/TEST/no_covid

mkdir: augmented: File exists


In [25]:
def wave_to_specgram_augment(folder_path):    
    file_list = os.listdir(folder_path)
    dataset = torch.empty(1)
    for file in file_list:
        if file != '.DS_Store':
            for i in [1/4, 1/2, 3/4, 1, 2]: # 1/4, 1/2, 3/4, 1, 2
                waveform, sample_rate = torchaudio.load(folder_path+'/'+file)
                waveform = torchaudio.transforms.Resample(sample_rate, sample_rate*i)(waveform)
                specgram = torchaudio.transforms.Spectrogram()(waveform)
                specgram_resize = torchvision.transforms.Resize((217,217))(specgram)
                plt.figure(frameon=False)
                plt.axis('off')
                specgram_resize += torch.ones(list(specgram_resize.shape))*1e-12
                plt.imshow(specgram_resize.log2()[0,:,:].numpy(), cmap='gray')
                plt.savefig('./augmented/'+folder_path.split('/')[-2]+'/'+folder_path.split('/')[-1]+'/'+file.strip('.wav')+'_'+str(i)+'.png', bbox_inches='tight',pad_inches=0)
                plt.close()

In [26]:
def wave_to_specgram(folder_path):    
    file_list = os.listdir(folder_path)
    dataset = torch.empty(1)
    for file in file_list:
        if file != '.DS_Store':
            waveform, sample_rate = torchaudio.load(folder_path+'/'+file)
            specgram = torchaudio.transforms.Spectrogram()(waveform)
            specgram_resize = torchvision.transforms.Resize((217,217))(specgram)
            plt.figure(frameon=False)
            plt.axis('off')
            specgram_resize += torch.ones(list(specgram_resize.shape))*1e-12
            plt.imshow(specgram_resize.log2()[0,:,:].numpy(), cmap='gray')
            plt.savefig('./augmented/'+folder_path.split('/')[-2]+'/'+folder_path.split('/')[-1]+'/'+file.strip('.wav')+'.png', bbox_inches='tight',pad_inches=0)
            plt.close()

In [30]:
train_covid_dir = './Cough_dataset/Labeled_audio/cough/TRAIN/covid'
train_nocovid_dir = './Cough_dataset/Labeled_audio/cough/TRAIN/no_covid'
test_covid_dir = './Cough_dataset/Labeled_audio/cough/TEST/covid'
test_nocovid_dir = './Cough_dataset/Labeled_audio/cough/TEST/no_covid'

wave_to_specgram_augment(train_covid_dir)
wave_to_specgram(train_nocovid_dir)
wave_to_specgram_augment(test_covid_dir)
wave_to_specgram(test_nocovid_dir)

RuntimeError: Error reading audio file: unknown length